In [1]:
# ruff: noqa: F401, E402, T201, T203, D103

from __future__ import annotations

import math
import sys
import time
from bisect import bisect_left, bisect_right
from collections import Counter, deque
from functools import cmp_to_key, lru_cache, reduce
from heapq import heapify, heappop, heappush, heappushpop, nsmallest
from importlib import reload
from itertools import (
    accumulate,
    chain,
    combinations,
    islice,
    pairwise,
    permutations,
    product,
    repeat,
    starmap,
    tee,
)
from math import comb, exp, factorial, inf, log, prod, sqrt
from operator import add, and_, contains, indexOf, itemgetter, mul, neg, or_, xor
from pprint import pprint
from typing import Callable, Iterable, List, Tuple

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
from multiprocess import Process

import arrays
import combinatorics
import graphs
import lists
import mathematics
import matrix
import parsing
import search
import sequences
import sets
import stack
import strings
import trees

for m in (
    arrays,
    combinatorics,
    graphs,
    lists,
    mathematics,
    matrix,
    parsing,
    search,
    sequences,
    sets,
    stack,
    strings,
    trees,
):
    reload(m)


from combinatorics import fib
from search import lower_index, upper_index
from sequences import find_if
from strings import splint

print()


In [83]:
def all_longest_common_subsequences(a: str, b: str) -> list:
    # determine the length of the lcs.
    ll = [[0, 0] for _ in range(len(a) + 1)]
    for j, i in product(range(len(b)), range(len(a))):
        ll[i + 1][1 - j & 1] = (
            1 + ll[i][j & 1]
            if a[i] == b[j]
            else max(ll[i + 1][j & 1], ll[i][1 - j & 1])
        )

    results = set()

    def bt(si: int, sj: int, s: list, l: int) -> None:
        if l == 0:
            results.add("".join(a[i] for i in s))
            return
        for i in range(si, len(a)):
            # find the next char in b that matches.
            j = b.find(a[i], sj)
            if j >= 0:
                s.append(i)
                bt(i + 1, j + 1, s, l - 1)
                s.pop()

    bt(0, 0, [], max(ll[-1]))
    return sorted(results)


In [84]:
all_longest_common_subsequences("cfb", "aceba")


[[0, 0], [1, 1], [1, 1], [2, 2]]


['cb']

In [85]:
all_longest_common_subsequences("baabaca", "abaaa")


[[0, 0], [1, 1], [2, 2], [3, 3], [3, 3], [3, 4], [3, 4], [4, 4]]


['aaaa', 'abaa', 'baaa']

In [86]:
all_longest_common_subsequences("abaaa", "baabaca")


[[0, 0], [1, 1], [2, 2], [3, 3], [3, 4], [4, 4]]


['aaaa', 'abaa', 'baaa']

In [87]:
all_longest_common_subsequences("abaaa", "abaaa")


[[0, 0], [1, 1], [2, 2], [3, 3], [4, 4], [4, 5]]


['abaaa']